In [1]:


import mobster as mb





Bad key "text.kerning_factor" on line 4 in
/Users/salvatore/miniconda3/envs/hmobster/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
data = mb.load_example_data("./")

#data = data["1:1"]

In [3]:
#data = { "1:1" : data}

In [4]:
fitted = mb.fit_mobster(data, 0,1, max_it = 1, lr = 0.001,  e = 0.00001, lrd_gamma = 0.1,   
        number_of_trials_k= 150, prior_lims_k=[100,10000])

Running MOBSTER on 3 karyotypes with 0 subclones.
Fitting a model with tail

Computing cluster assignements.
tensor([0.9120], grad_fn=<SelectBackward>)
tensor([0.4800], grad_fn=<SelectBackward>)
Computing information criteria.
tensor([0.9120], grad_fn=<SelectBackward>)
tensor([0.4800], grad_fn=<SelectBackward>)
tensor([0.9120], grad_fn=<SelectBackward>)
tensor([0.4800], grad_fn=<SelectBackward>)
tensor([0.9120], grad_fn=<SelectBackward>)
tensor([0.4800], grad_fn=<SelectBackward>)
tensor([0.9120], grad_fn=<SelectBackward>)
tensor([0.4800], grad_fn=<SelectBackward>)
Done!


ELBO: 3705.178057554  : 100%|██████████| 1/1 [00:00<00:00, 32.57it/s]


In [7]:
mb.plot_results(data, fitted, output= "cane.png", drivers=None)

In [ ]:
fitted["model_parameters"]["1:1"]



0.01

1